In [ ]:
import torch
torch.manual_seed(0)

from chop.ir.graph.mase_graph import MaseGraph

from chop.passes.graph.analysis import (
    init_metadata_analysis_pass,
    add_common_metadata_analysis_pass,
    add_hardware_metadata_analysis_pass,
    report_node_type_analysis_pass,
)

from chop.passes.graph.transforms import (
    emit_verilog_top_transform_pass,
    emit_internal_rtl_transform_pass,
    emit_bram_transform_pass,
    emit_cocotb_transform_pass,
    quantize_transform_pass,
)

from chop.tools.logger import set_logging_verbosity

set_logging_verbosity("debug")

import toml
import torch
import torch.nn as nn
import os
os.environ["PATH"] = "/homes/oa321/verilator/verilator/bin:" + os.environ["PATH"]
!verilator

: 

In [3]:
class MLP(torch.nn.Module):
    """
    Toy FC model for digit recognition on MNIST
    """

    def __init__(self) -> None:
        super().__init__()

        self.fc1 = nn.Linear(4, 8)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1, end_dim=-1)
        x = torch.nn.functional.relu(self.fc1(x))
        return x

In [4]:
mlp = MLP()
mg = MaseGraph(model=mlp)

# Provide a dummy input for the graph so it can use for tracing
batch_size = 1
x = torch.randn((batch_size, 2, 2))
dummy_in = {"x": x}

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(
    mg, {"dummy_in": dummy_in, "add_value": False}
)

DEBUG    graph():
    %x : [num_users=1] = placeholder[target=x]
    %flatten : [num_users=1] = call_function[target=torch.flatten](args = (%x,), kwargs = {start_dim: 1, end_dim: -1})
    %fc1 : [num_users=1] = call_module[target=fc1](args = (%flatten,), kwargs = {})
    %relu : [num_users=1] = call_function[target=torch.nn.functional.relu](args = (%fc1,), kwargs = {inplace: False})
    return relu


In [5]:
config_file = os.path.join(
    os.path.abspath(""),
    "..",
    "..",
    "configs",
    "tests",
    "quantize",
    "fixed.toml",
)
with open(config_file, "r") as f:
    quan_args = toml.load(f)["passes"]["quantize"]
mg, _ = quantize_transform_pass(mg, quan_args)

_ = report_node_type_analysis_pass(mg)

# Update the metadata
for node in mg.fx_graph.nodes:
    for arg, arg_info in node.meta["mase"]["common"]["args"].items():
        if isinstance(arg_info, dict):
            arg_info["type"] = "fixed"
            arg_info["precision"] = [8, 3]
    for result, result_info in node.meta["mase"]["common"]["results"].items():
        if isinstance(result_info, dict):
            result_info["type"] = "fixed"
            result_info["precision"] = [8, 3]

INFO     Inspecting graph [add_common_node_type_analysis_pass]
INFO     
Node name    Fx Node op     Mase type            Mase op      Value type
-----------  -------------  -------------------  -----------  ------------
x            placeholder    placeholder          placeholder  NA
flatten      call_function  implicit_func        flatten      float
fc1          call_module    module_related_func  linear       fixed
relu         call_function  module_related_func  relu         fixed
output       output         output               output       NA


In [6]:
mg, _ = add_hardware_metadata_analysis_pass(mg)

In [7]:
mg, _ = emit_verilog_top_transform_pass(mg)
mg, _ = emit_internal_rtl_transform_pass(mg)

INFO     Emitting Verilog...
INFO     Emitting internal components...


In [8]:
mg, _ = emit_bram_transform_pass(mg)

INFO     Emitting BRAM...
DEBUG    Emitting DAT file for node: fc1, parameter: weight
DEBUG    ROM module weight successfully written into /homes/oa321/.mase/top/hardware/rtl/fc1_weight_source.sv
DEBUG    Init data weight successfully written into /homes/oa321/.mase/top/hardware/rtl/fc1_weight_rom.dat
DEBUG    Emitting DAT file for node: fc1, parameter: bias
DEBUG    ROM module bias successfully written into /homes/oa321/.mase/top/hardware/rtl/fc1_bias_source.sv
DEBUG    Init data bias successfully written into /homes/oa321/.mase/top/hardware/rtl/fc1_bias_rom.dat


In [9]:
mg, _ = emit_cocotb_transform_pass(mg)

INFO     Emitting testbench...


In [12]:
from chop.actions import simulate
import shutil

executable = shutil.which("verilator")
print(f"Exec: {executable}")

simulate(skip_build=False, skip_test=False)

Exec: /homes/oa321/verilator/verilator/bin/verilator
INFO: Running command perl /homes/oa321/verilator/verilator/bin/verilator -cc --exe -Mdir /vol/bitbucket/oa321/mase/tasks/tutorial6/sim_build -DCOCOTB_SIM=1 --top-module top --vpi --public-flat-rw --prefix Vtop -o top -LDFLAGS '-Wl,-rpath,/vol/bitbucket/oa321/mase/venv/lib/python3.11/site-packages/cocotb/libs -L/vol/bitbucket/oa321/mase/venv/lib/python3.11/site-packages/cocotb/libs -lcocotbvpi_verilator' -Wno-fatal -Wno-lint -Wno-style --trace-fst --trace-structs --trace-depth 3 -I/homes/oa321/.mase/top/hardware/rtl -I/vol/bitbucket/oa321/mase/src/mase_components/normalization_layers/rtl -I/vol/bitbucket/oa321/mase/src/mase_components/memory/rtl -I/vol/bitbucket/oa321/mase/src/mase_components/activation_layers/rtl -I/vol/bitbucket/oa321/mase/src/mase_components/cast/rtl -I/vol/bitbucket/oa321/mase/src/mase_components/interface/rtl -I/vol/bitbucket/oa321/mase/src/mase_components/convolution_layers/rtl -I/vol/bitbucket/oa321/mase/src/m

INFO     Build finished. Time taken: 0.30s


- V e r i l a t i o n   R e p o r t: Verilator 5.033 devel rev v5.032-62-g6b4183632
- Verilator: Built from 0.000 MB sources in 0 modules, into 0.000 MB in 0 C++ files needing 0.000 MB
- Verilator: Walltime 0.126 s (elab=0.000, cvt=0.000, bld=0.000); cpu 0.005 s on 1 threads; alloced 18.645 MB
INFO: Running command make -C /vol/bitbucket/oa321/mase/tasks/tutorial6/sim_build -f Vtop.mk in directory /vol/bitbucket/oa321/mase/tasks/tutorial6/sim_build
make: Entering directory '/vol/bitbucket/oa321/mase/tasks/tutorial6/sim_build'
make: Nothing to be done for 'default'.
make: Leaving directory '/vol/bitbucket/oa321/mase/tasks/tutorial6/sim_build'
INFO: Running command /vol/bitbucket/oa321/mase/tasks/tutorial6/sim_build/top in directory /vol/bitbucket/oa321/mase/tasks/tutorial6/sim_build


TypeError: expected str, bytes or os.PathLike object, not NoneType